# Fine-Tune

In [1]:
import joblib
import numpy as np

housing_prepared = joblib.load("data/housing_prepared.pkl")
housing_labels = joblib.load("data/housing_labels.pkl")
housing = joblib.load("data/housing_raw.pkl") 
strat_train_set = joblib.load("data/strat_train_set.pkl")
strat_test_set = joblib.load("data/strat_test_set.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'data/housing_prepared.pkl'

In [ ]:
class DataFrameSelector(BaseEstimator, TransformerMixin):  # 재정의
    def __init__(self, attribute_names):  # 선택할 열 이름 목록을 초기화
        self.attribute_names = attribute_names
    def fit(self, X, y=None):  # fit 메서드 (여기선 학습 필요 없음)
        return self
    def transform(self, X):  # 지정된 열만 NumPy 배열 형태로 추출하여 반환
        return X[self.attribute_names] # .values 제거

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin  # 재정의

# 열 인덱스 설정
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6  # 순서대로 total_rooms, total_bedrooms, population, households 열의 인덱스

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):  # 파생 특성 생성용 사용자 정의 변환기 클래스 정의
    def __init__(self, add_bedrooms_per_room=True):  # bedrooms_per_room 특성을 추가할지 여부를 설정하는 하이퍼파라미터
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):  # fit 메서드는 아무 작업 없이 self 반환 (필수 메서드)
        return self
    def transform(self, X, y=None):  # transform 메서드에서 새로운 파생 특성들을 계산하여 기존 데이터에 추가
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]  # 세 개의 파생 특성 추가
        else:
            return np.c_[X, rooms_per_household, population_per_household]  # 두 개의 파생 특성만 추가

In [ ]:
full_pipeline = joblib.load("models/full_pipeline.pkl")

## Grid Search

하이퍼파라미터를 손으로 하나씩 조정하면서 좋은 조합을 찾는 대신 
Scikit-Learn의 `GridSearchCV` 사용
>이 도구에 어떤 하이퍼파라미터들을 실험할지,  
>그리고 각 하이퍼파라미터마다 어떤 값을 시도할지를 알려주기만 하면  
>모든 가능한 하이퍼파라미터 조합을 교차 검증을 통해 평가해 줌

In [ ]:
from sklearn.model_selection import GridSearchCV  # 하이퍼파라미터 튜닝을 위한 GridSearchCV 임포트
from sklearn.ensemble import RandomForestRegressor  # 랜덤 포레스트 회귀 모델 임포트

param_grid = [  # 탐색할 하이퍼파라미터 조합 정의
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor()  # 모델 객체 생성

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,  # 그리드 탐색 객체 생성
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)  # 전체 훈련 세트로 학습 및 교차 검증

어떤 하이퍼파라미터의 값을 전혀 모르겠을 때는,  
10의 거듭제곱(또는 더 세밀하게 탐색하고 싶다면 더 작은 수)을 순서대로 시도해보는 것이 간단한 접근법(이 예제에서 `n_estimators` 하이퍼파라미터)

이 `param_grid`는 Scikit-Learn에게 첫 번째 딕셔너리에 지정된 `n_estimators`와 `max_features` 하이퍼파라미터 값의   
모든 3 × 4 = 12가지 조합을 먼저 평가하라고 지시(이 하이퍼파라미터들 의미는 7장에서)  
그런 다음, 두 번째 딕셔너리에 있는 하이퍼파라미터 조합 2 × 3 = 6가지를 평가하는데,   
이때는 기본값인 `bootstrap=True` 대신 `bootstrap=False`로 설정해서 진행  

In [ ]:
grid_search.best_params_  # 가장 좋은 하이퍼파라미터 조합 출력

6과 30은 평가된 값들 중 최대값이므로, 더 높은 값을 사용해 다시 탐색해보는 것이 좋음(점수가 계속해서 개선될 수 있기 때문)

In [ ]:
grid_search.best_estimator_  # 가장 성능이 좋은 모델 출력

GridSearchCV가 refit=True로 초기화되면(기본값),  
교차 검증을 통해 최적의 추정기를 찾은 후 전체 학습 세트로 다시 훈련  
이는 일반적으로 좋은 방법인데, 더 많은 데이터를 제공하면 성능이 향상될 가능성이 높기 때문

In [ ]:
cvres = grid_search.cv_results_  # 교차 검증 결과 저장
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)  # RMSE와 하이퍼파라미터 조합 출력

이 예제에서는 `max_features` 하이퍼파라미터를 6으로, `n_estimators` 하이퍼파라미터를 30으로 설정함으로써 최적의 해를 얻음  
이 조합의 RMSE 점수는 50,159로, 이전에 기본 하이퍼파라미터 값을 사용했을 때의 점수인 50,182보다 약간 더 좋음???

데이터 준비 단계 중 일부도 하이퍼파라미터로 다룰 수 있다는 점을 잊지 말아야 함
>예를 들어, 그리드 서치를 통해 확신이 없던 특성(예: `CombinedAttributesAdder` 변환기의 `add_bedrooms_per_room` 하이퍼파라미터)을 추가할지 말지를 자동으로 찾아낼 수 있음  
>이와 유사하게 이상치 처리, 결측 특성 처리, 특성 선택 등에도 최적의 방식을 자동으로 찾아내는 데 사용할 수 있음

In [ ]:
pd.DataFrame(grid_search.cv_results_)

## Randomized Search(랜덤 탐색)

그리드 서치 방식은 앞선 예제처럼 비교적 적은 수의 조합을 탐색할 때는 괜찮지만,  
하이퍼파라미터 탐색 공간이 클 경우에는 `RandomizedSearchCV`를 사용하는 것이 더 나은 경우가 많음  
이 클래스는 `GridSearchCV`와 거의 같은 방식으로 사용할 수 있지만, 가능한 모든 조합을 시도하는 대신,   
각 반복에서 하이퍼파라미터마다 임의의 값을 선택해 지정된 수의 무작위 조합을 평가함  

장점  
1. 예를 들어 랜덤 탐색을 1,000회 반복하도록 설정하면, 각 하이퍼파라미터에 대해 1,000개의 서로 다른 값을 탐색하게 됨
(그리드 서치 방식에서는 하이퍼파라미터당 몇 개의 값만 시도됨)  
2. 반복 횟수만 설정함으로써, 하이퍼파라미터 탐색에 얼마만큼의 계산 자원을 할당할지 더 쉽게 제어할 수 있다.

## Ensemble Methods(앙상블 기법)

시스템을 미세 조정하는 또 다른 방법 -> 성능이 좋은 여러 모델을 결합하는 것  
이 집단(또는 “앙상블”)은 개별 모델 중 가장 좋은 것보다도 성능이 더 좋을 때가 많음  
(`Random Forest`가 개별 `Decision Tree`보다 더 좋은 성능을 보이는 것처럼)  
특히 개별 모델들이 서로 매우 다른 유형의 오류를 범할 때 효과가 큼

이 주제는 7장에서 더 자세히 다뤄짐


## Analyze the Best Models and Their Errors

최고의 모델들을 살펴보면 문제에 대한 좋은 통찰을 얻는 경우가 많음  
예를 들어, `RandomForestRegressor`는 정확한 예측을 위해 각 특성이 얼마나 중요한지를 나타내는 상대적 중요도를 제공할 수 있음

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_  # 특성 중요도 추출

In [ ]:
feature_importances

In [ ]:
from sklearn.preprocessing import OneHotEncoder

housing_cat = housing[["ocean_proximity"]]
cat_encoder = OneHotEncoder(sparse_output=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

cat_one_hot_attribs = list(cat_encoder.get_feature_names_out())

In [ ]:
# 수치형 열만 추출하고 열 이름 리스트 저장
housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]  # 파생 변수 이름
cat_one_hot_attribs = list(cat_encoder.get_feature_names_out())  # 원-핫 인코딩된 범주형 변수 이름
attributes = num_attribs + extra_attribs + cat_one_hot_attribs  # 전체 특성 이름 합치기

In [ ]:
sorted(zip(feature_importances, attributes), reverse=True)  # 특성 중요도 내림차순 정렬

이 정보를 바탕으로 덜 유용한 특성들을 제거해볼 수도 있음  
(예: `ocean_proximity` 카테고리 중 실제로 유용한 것은 하나뿐인 것 같으므로 나머지는 제거해볼 수 있음)  
또한 시스템이 어떤 구체적인 오류를 범하는지도 살펴보고, 왜 그런 오류가 발생하는지,  
그리고 이를 어떻게 해결할 수 있을지 고민해봐야 함(추가적인 특성을 더하거나, 정보가 없는 특성을 제거하거나, 이상치를 정리하는 등)

## Evaluate Your System on the Test Set

최종 모델을 테스트 세트에서 평가
- 테스트 세트에서 입력값과 레이블을 가져오고, 
- 전체 파이프라인에서 transform()을 호출해 데이터를 변환한 다음(테스트 세트를 학습시키지 않기 위해 fit_transform()이 아니라 transform()을 호출해야), 
- 테스트 세트에서 최종 모델을 평가

In [ ]:
from sklearn.metrics import mean_squared_error  # RMSE 계산을 위한 모듈 임포트

final_model = grid_search.best_estimator_  # 최종 모델 지정

X_test = strat_test_set.drop("median_house_value", axis=1)  # 테스트 입력 특성
y_test = strat_test_set["median_house_value"].copy()  # 테스트 타깃 레이블

X_test_prepared = full_pipeline.transform(X_test)  # 전처리 파이프라인 적용
final_predictions = final_model.predict(X_test_prepared)  # 예측 수행

final_mse = mean_squared_error(y_test, final_predictions)  # MSE 계산
final_rmse = np.sqrt(final_mse)  # RMSE 계산
print(final_rmse)  # RMSE 출력

어떤 경우에는 일반화 오차에 대한 이러한 점 추정값만으로는 실제로 모델을 배포할지 확신하기에 충분하지 않을 수 있다. 
예를 들어, 현재 운영 중인 모델보다 단지 0.1%만 더 나은 경우라면, 이 추정값이 얼마나 정확한지를 알고 싶을 수도 있다.

이를 위해 `scipy.stats.t.interval()`을 사용해 일반화 오차에 대한 95% 신뢰 구간을 계산할 수 있다.

In [ ]:
joblib.dump(grid_search, "models/grid_search.pkl")